# RSG Week 5

This week, we'll be using different data structures to simulate a hospital. Please note that due to the breadth and depth of the topic, we will be operating under the assumption that you know what these structures are (by definition), and are interested in applying them. To refresh yourself on the definitions, the course notes are always available from Professor Lanthier's website.

We'll start off with some helper classes. You can look inside these if you're curious, **but they're not essential material**. Click [here](#Demo) to skip the helper classes.

## Helper Classes

### Patient Class

In [1]:
public class Patient implements Comparable<Patient>{
    String name;
    int age;
    String illness;
    double severity;

    boolean deceased;
    boolean released;

    public Patient(){

    }

    public Patient(String n, int a, String i){
        name = n;
        age = a;
        illness = i;
    }

    /*
        OVERRIDING
     */

    public int compareTo(Patient p){
        return (int)((p.severity - severity)*100);
    }

    public String toString(){
        String d = deceased ? (" (D)") : ("");
        return String.format("Patient: %-25s Age: %-5d Illness: %-15s Severity: %-41.2f", name+d, age, illness, severity);
    }

    @Override
    public boolean equals(Object obj) {
        return obj instanceof Patient && hashCode() == obj.hashCode();
    }

    @Override
    public int hashCode() {
        return name.hashCode()+((Integer)age).hashCode();
    }
}

com.twosigma.beaker.javash.bkr5bd46cbd.Patient

### Triage Class

In [2]:
import java.util.PriorityQueue;
import java.util.Stack;

public class Triage extends PriorityQueue<Patient>{


    public Triage(){

    }

    public void addToQueue(Patient p){
        add(p);
    }

    public Stack<Patient> clearTriage(){
        Stack<Patient> toBeAdmitted = new Stack<Patient>();

        for(Patient p : this){
            toBeAdmitted.add(p);
        }
        clear();

        return toBeAdmitted;
    }
}

com.twosigma.beaker.javash.bkr5bd46cbd.Triage

### Hospital class

In [3]:
import java.util.*;

public class Hospital {
    HashSet<Patient> patients = new HashSet<Patient>();
    HashMap<Date, ArrayList<Patient>> visits = new HashMap<Date, ArrayList<Patient>>();

    LinkedList<Patient> beds = new LinkedList<Patient>();

    HashMap<String, Double> illnesses = new HashMap<String, Double>(); // illness, severity (0-10)
    Triage triage = new Triage();

    public Hospital(){

    }

    public HashSet<Patient> getPatients(){
        return patients;
    }

    public void sendToTriage(Date d, Patient p){
        p.severity = diagnose(p);
        triage.addToQueue(p);

        if(visits.containsKey(d)){
            for(Date date: visits.keySet()){
                if(date.equals(d)){
                    visits.get(date).add(p);
                }
            }
        }
        else {
            ArrayList<Patient> newList = new ArrayList<Patient>();
            newList.add(p);
            visits.put(d, newList);
        }
    }

    public double severity(String illness){
        int n = 0;
        int d = 0;
        for(ArrayList<Patient> plist : visits.values()){
            for(Patient p : plist){
                if(illness.equals(p.illness)){
                    n++;
                    if(p.deceased){
                        d++;
                    }
                    else if(p.released){
                        d--;
                    }
                }
            }
        }
        if(n > 0){
            return (double)d/n;
        }
        return 0;
    }

    public void updateDiagnoses(String illness){
        if(illnesses.containsKey(illness)){
            double update = illnesses.get(illness)+severity(illness);
            illnesses.put(illness, update);
        }
        else {
            illnesses.put(illness, 5.0);
        }

        for(Patient p : patients){
            p.severity = diagnose(p);
        }
        
        for(Patient p : triage){
            p.severity = diagnose(p);
        }
    }

    public double diagnose(Patient p){
        if(illnesses.containsKey(p.illness)) {
            return illnesses.get(p.illness);
        }
        illnesses.put(p.illness, 5.0);
        return 5.0;
    }

    public void admitPatients(){
        Stack<Patient> toBeAdmitted = triage.clearTriage();
        while(!toBeAdmitted.empty()){
            Patient p = toBeAdmitted.pop();
            patients.add(p);
            beds.add(p);
        }
    }

    public Patient findPatient(String name){
        for(Patient p : patients){
            if(name.equals(p.name)){
                return p;
            }
        }
        return null;
    }

    public void patientDeceased(String name){
        Patient p = findPatient(name);
        if(p != null){
            p.deceased = true;
            updateDiagnoses(p.illness);
            beds.remove(p);
        }
    }

    public void patientReleased(String name){
        Patient p = findPatient(name);
        if(p != null){
            p.released = true;
            updateDiagnoses(p.illness);
            beds.remove(p);
        }
    }

    public double getIllnessSeverity(String illness){
        return illnesses.get(illness);
    }

    public Integer getBed(Patient p){
        Integer bed = beds.indexOf(p)+1;
        if(bed != -1){
            return bed;
        }
        return null;
    }

    public void showIllnesses(){
        for(String s: illnesses.keySet()){
            System.out.println(String.format("Illness: %-30s Severity: %1.2f", s, illnesses.get(s)));
        }
    }

    public void showTriage(){
        if(triage.size() == 0){
            System.out.println("Exception: Triage empty");
        }
        for(Patient p : triage){
            System.out.println(p);
        }
    }

    public void showAdmitted(){
        for(Patient p : beds){
            System.out.println(String.format("Bed %02d: %s", getBed(p), p));
        }
    }

    public void showPreviousVisits(String name){
        Patient pat = findPatient(name);
        if(pat != null){
            System.out.println(String.format("Visits by %s (Age %d)", pat.name, pat.age));
            for(Date d : visits.keySet()){
                ArrayList<Patient> plist = visits.get(d);
                for(Patient p : plist){
                    if(p.equals(pat)){
                        System.out.println(String.format("Date: %-30s", d.toString()));
                    }
                }
            }
        }
    }
    
    public void showPatientRecords(){
        for(Patient p : patients) {
            System.out.println(String.format("Patient: %-25s Age: %-5d", p.name, p.age));
        }
    }
}

com.twosigma.beaker.javash.bkr5bd46cbd.Hospital

### Sample patients

In [4]:
import java.util.ArrayList;
import java.util.Date;
import java.util.GregorianCalendar;
import java.util.HashMap;

public class Ex {
    public static Hospital h = new Hospital();
    
    public static HashMap<Date, ArrayList<Patient>> example1(){
        HashMap<Date, ArrayList<Patient>> patients = new HashMap<Date, ArrayList<Patient>>();
        ArrayList<Patient> patientList = new ArrayList<Patient>();
        Date date = new GregorianCalendar(2018, GregorianCalendar.JANUARY, 5).getTime();

        patientList.add(new Patient("Mary Measles", 47, "Chest pains"));
        patientList.add(new Patient("Fred Ziffle", 56, "Headaches" ));
        patientList.add(new Patient("Blaine Little", 92, "Heart attack"));
        patientList.add(new Patient("Bob Pointer", 86, "Heart attack"));
        patientList.add(new Patient("Greg Martell", 56, "Heart attack"));

        patients.put(date, patientList);
        return patients;
    }

    public static HashMap<Date, ArrayList<Patient>> example2(){
        HashMap<Date, ArrayList<Patient>> patients = new HashMap<Date, ArrayList<Patient>>();
        ArrayList<Patient> patientList = new ArrayList<Patient>();
        Date date = new GregorianCalendar(2018, GregorianCalendar.JANUARY, 6).getTime();

        patientList.add(new Patient("Martha Stewart", 65, "Heart attack"));
        patientList.add(new Patient("John Junior", 26, "Weak knees"));
        patientList.add(new Patient("Jane Doe", 34, "Flu"));
        patientList.add(new Patient("Herb Platt", 32, "Nausea"));
        patientList.add(new Patient("Donald Duck", 46, "Chest pains"));

        patients.put(date, patientList);
        return patients;
    }

    public static HashMap<Date, ArrayList<Patient>> example3(){
        HashMap<Date, ArrayList<Patient>> patients = new HashMap<Date, ArrayList<Patient>>();
        ArrayList<Patient> patientList = new ArrayList<Patient>();
        Date date = new GregorianCalendar(2018, GregorianCalendar.JANUARY, 7).getTime();

        patientList.add(new Patient("Mary Measles", 47, "Nausea"));
        patientList.add(new Patient("Lark Manthier", 48, "Flu"));

        patients.put(date, patientList);
        return patients;
    }
    
    public static void triagePatients(int num){
        HashMap<Date, ArrayList<Patient>> patients = new HashMap();
        switch (num){
            case 1: patients = example1();
                break;
            case 2: patients = example2();
                break;
            case 3: patients = example3();
                break;
        }
        
        for(Date d : patients.keySet()){
            ArrayList<Patient> plist = patients.get(d);
            for(Patient p : plist){
                h.sendToTriage(d, p);
            }
        }
    }
}

com.twosigma.beaker.javash.bkr5bd46cbd.Ex

For reference, here are the key points to know about these classes:

* Patients have names, ages, deceased and released statuses, and illnesses
    * We say patients are equal if their _name_ and _age_ are equal - not their illness (more on this later)
* Hospitals have:
    * Patient records
        * These records are stored as a HashSet. This is because we don't need a new record generated for each patient every time they visit the hospital - we already have their information (no duplicates). 
    * Visit records
        * These records are stored as a HashMap - mapping a date to a list of patients who visited that date. These lists can contain duplicates, as patients can visit more than once.
    * Records of which patient is in which bed
        * These records are stored as a Linked List - we do this because our hospital's beds are sorted linearly.
    * Illness records
        * These records are stored as a HashMap - mapping a String (the illness) to an int (the severity of the illness). We do this to quickly access illness severities, and because we don't want duplicate illnesses. 
    * A triage system
        * Triages are great examples of Queues - specifically, priority queues (more on this shortly).

## Demo

### Triaging

When you go to a hospital, you start off with by going through the triage process. Likewise, we'll put our first set of exapmle patients through the triage process. How do we do this? 

triagePatients() is simply a small helper method I've written to generate examples. What's important is that it calls a method in our hospital class, sendToTriage(). This takes a Date `d` and a Patient `p`. Here's how we begin:

```java
public void sendToTriage(Date d, Patient p){
        p.severity = diagnose(p);
        triage.addToQueue(p);
    }
```

Let's take a look inside `diagnose(p)`:
```java
public double diagnose(Patient p){
        if(illnesses.containsKey(p.illness)) {
            return illnesses.get(p.illness);
        }
        illnesses.put(p.illness, 5.0);
        return 5.0;
    }
```

First, we check if our hashmap contains the patient's illness. If it does, we simply return the severity. If not, we add it to the hashmap with some default value (let's do `5.0`).



Now let's jump into the rest of the code. We use similar logic with our `visits` hashmap. We check if the date already exists: if it does, we add the patient to the list of visits that day; if it doesn't, we add the date with a 1-patient list (as that patient is the first visitor). That's all that's happening here!

```java
public void sendToTriage(Date d, Patient p){
        p.severity = diagnose(p);
        triage.addToQueue(p);

        if(visits.containsKey(d)){
            for(Date date: visits.keySet()){
                if(date.equals(d)){
                    visits.get(date).add(p);
                }
            }
        }
        else {
            ArrayList<Patient> newList = new ArrayList<Patient>();
            newList.add(p);
            visits.put(d, newList);
        }
    }
```

In [5]:
System.out.println("Triage some patients:");

Ex.triagePatients(1);
Ex.h.showTriage();

Triage some patients:
Patient: Mary Measles              Age: 47    Illness: Chest pains     Severity: 5.00                                     
Patient: Fred Ziffle               Age: 56    Illness: Headaches       Severity: 5.00                                     
Patient: Blaine Little             Age: 92    Illness: Heart attack    Severity: 5.00                                     
Patient: Bob Pointer               Age: 86    Illness: Heart attack    Severity: 5.00                                     
Patient: Greg Martell              Age: 56    Illness: Heart attack    Severity: 5.00                                     


null

Great, our patients have been triaged. As a triage is a priority queue, they should be sorted by the severity of the illnesses. Let's take a look:

In [6]:
System.out.println("Illness severities:");
Ex.h.showIllnesses();

Illness severities:
Illness: Chest pains                    Severity: 5.00
Illness: Headaches                      Severity: 5.00
Illness: Heart attack                   Severity: 5.00


null

Of course, our triage isn't really sorted right now - these are all new illnesses, so they all begin at the default values.

### Admitting Patients

Next, let's admit our patients from triage. Because our triage is a priority queue, we know that the order in which they'll be admitted is in order of how severe their illnesses are.

We can do this using a Stack. We call our triage's clearTriage() method:

```java
public Stack<Patient> clearTriage(){
        Stack<Patient> toBeAdmitted = new Stack<Patient>();

        for(Patient p : this){
            toBeAdmitted.add(p);
        }
        clear();

        return toBeAdmitted;
    }
```

We iterate through our triage queue (`this`) and add each Patient to the stack. We then clear the queue and return the Stack.

Note: We use this strategy because we can't simply add to our Stack and remove them from the queue, inside the for loop. This will be a _concurrent modification exception_. 

With our Stack, we pop each patient, and add them to our records. 

```java
public void admitPatients(){
        Stack<Patient> toBeAdmitted = triage.clearTriage();
        while(!toBeAdmitted.empty()){
            Patient p = toBeAdmitted.pop();
            patients.add(p);
            beds.add(p);
        }
    }
```

In [7]:
Ex.h.admitPatients();
Ex.h.showAdmitted();

Bed 01: Patient: Greg Martell              Age: 56    Illness: Heart attack    Severity: 5.00                                     
Bed 02: Patient: Bob Pointer               Age: 86    Illness: Heart attack    Severity: 5.00                                     
Bed 03: Patient: Blaine Little             Age: 92    Illness: Heart attack    Severity: 5.00                                     
Bed 04: Patient: Fred Ziffle               Age: 56    Illness: Headaches       Severity: 5.00                                     
Bed 05: Patient: Mary Measles              Age: 47    Illness: Chest pains     Severity: 5.00                                     


null

And now our triage should be empty:

In [8]:
Ex.h.showTriage();

Exception: Triage empty


null

Let's add some more patients.

In [9]:
Ex.triagePatients(2);
Ex.h.showTriage();

Patient: Martha Stewart            Age: 65    Illness: Heart attack    Severity: 5.00                                     
Patient: John Junior               Age: 26    Illness: Weak knees      Severity: 5.00                                     
Patient: Jane Doe                  Age: 34    Illness: Flu             Severity: 5.00                                     
Patient: Herb Platt                Age: 32    Illness: Nausea          Severity: 5.00                                     
Patient: Donald Duck               Age: 46    Illness: Chest pains     Severity: 5.00                                     


null

We see we've got some new illnesses.

In [10]:
Ex.h.showIllnesses();

Illness: Chest pains                    Severity: 5.00
Illness: Headaches                      Severity: 5.00
Illness: Nausea                         Severity: 5.00
Illness: Weak knees                     Severity: 5.00
Illness: Heart attack                   Severity: 5.00
Illness: Flu                            Severity: 5.00


null

Unfortunately, one of our patients has died from his heart attack. Let's see how this affects the severity:

In [11]:
Ex.h.patientDeceased("Greg Martell");
System.out.println("Heart attack severity:\t"+Ex.h.getIllnessSeverity("Heart attack"));

Heart attack severity:	5.25


null

We handle patient death or release with a fairly simple method. We simply find them, set their status to deceased, update our illness severities (content isn't relevant), and remove them from our linked list of beds. We do the same thing for releasing patients, but we set their "released" status instead.

```java
public void patientDeceased(String name){
        Patient p = findPatient(name);
        if(p != null){
            p.deceased = true;
            updateDiagnoses(p.illness);
            beds.remove(p);
        }
    }
```

Now, let's build on this a bit. Let's release all patients with chest pains. We'll write our stack, use an enhanced for loop to add all patients with chest pains, and then release them with a while loop. Once again, _we cannot remove them in our for loop - this is a concurrent modification error_. 

In [12]:
import java.util.Stack;

Stack<Patient> toBeReleased = new Stack<Patient>();
for(Patient p : Ex.h.getPatients()){
    if(p.illness.equals("Chest pains"))
        toBeReleased.push(p);
}
while(!toBeReleased.empty()){
    Ex.h.patientReleased(toBeReleased.pop().name);
}

Ex.h.showAdmitted();

Bed 01: Patient: Bob Pointer               Age: 86    Illness: Heart attack    Severity: 5.25                                     
Bed 02: Patient: Blaine Little             Age: 92    Illness: Heart attack    Severity: 5.25                                     
Bed 03: Patient: Fred Ziffle               Age: 56    Illness: Headaches       Severity: 5.00                                     


null

Now, let's take another look at our triage. So far:

* A patient has died of a heart attack
* Several patients have survived their chest pains and released

So, our triage should now be sorted by illness severities. Let's see what these are:

In [13]:
Ex.h.showIllnesses();

Illness: Chest pains                    Severity: 4.50
Illness: Headaches                      Severity: 5.00
Illness: Nausea                         Severity: 5.00
Illness: Weak knees                     Severity: 5.00
Illness: Heart attack                   Severity: 5.25
Illness: Flu                            Severity: 5.00


null

And our triage:

In [14]:
Ex.h.showTriage();

Patient: Martha Stewart            Age: 65    Illness: Heart attack    Severity: 5.25                                     
Patient: John Junior               Age: 26    Illness: Weak knees      Severity: 5.00                                     
Patient: Jane Doe                  Age: 34    Illness: Flu             Severity: 5.00                                     
Patient: Herb Platt                Age: 32    Illness: Nausea          Severity: 5.00                                     
Patient: Donald Duck               Age: 46    Illness: Chest pains     Severity: 4.50                                     


null

It worked! They're sorted in order of their illness severity. How did we accomplish this?

### Sorting by comparison

We do this by overriding the compareTo method. As professor Lanthier demonstrated, we take in a Patient object, and compare their severity to its own.

```java
public int compareTo(Patient p){
        return (int)((p.severity - severity)*100);
    }
```

Let's admit all the patients in our triage.

In [15]:
Ex.h.admitPatients();
Ex.h.showAdmitted();

Bed 01: Patient: Bob Pointer               Age: 86    Illness: Heart attack    Severity: 5.25                                     
Bed 02: Patient: Blaine Little             Age: 92    Illness: Heart attack    Severity: 5.25                                     
Bed 03: Patient: Fred Ziffle               Age: 56    Illness: Headaches       Severity: 5.00                                     
Bed 04: Patient: Donald Duck               Age: 46    Illness: Chest pains     Severity: 4.50                                     
Bed 05: Patient: Herb Platt                Age: 32    Illness: Nausea          Severity: 5.00                                     
Bed 06: Patient: Jane Doe                  Age: 34    Illness: Flu             Severity: 5.00                                     
Bed 07: Patient: John Junior               Age: 26    Illness: Weak knees      Severity: 5.00                                     
Bed 08: Patient: Martha Stewart            Age: 65    Illness: Heart attack    Seve

null

And now admit our last example:

In [16]:
Ex.triagePatients(3);
Ex.h.admitPatients();
Ex.h.showAdmitted();

Bed 01: Patient: Bob Pointer               Age: 86    Illness: Heart attack    Severity: 5.25                                     
Bed 02: Patient: Blaine Little             Age: 92    Illness: Heart attack    Severity: 5.25                                     
Bed 03: Patient: Fred Ziffle               Age: 56    Illness: Headaches       Severity: 5.00                                     
Bed 04: Patient: Donald Duck               Age: 46    Illness: Chest pains     Severity: 4.50                                     
Bed 05: Patient: Herb Platt                Age: 32    Illness: Nausea          Severity: 5.00                                     
Bed 06: Patient: Jane Doe                  Age: 34    Illness: Flu             Severity: 5.00                                     
Bed 07: Patient: John Junior               Age: 26    Illness: Weak knees      Severity: 5.00                                     
Bed 08: Patient: Martha Stewart            Age: 65    Illness: Heart attack    Seve

null

It looks like Mary is back, and this time she has nausea; not a good week for Mary. We can look through her previous visits:

In [17]:
Ex.h.showPreviousVisits("Mary Measles");

Visits by Mary Measles (Age 47)
Date: Fri Jan 05 00:00:00 EST 2018  
Date: Sun Jan 07 00:00:00 EST 2018  


null

However, when we look through our record of patients, there exists only one instance of her!

In [18]:
Ex.h.showPatientRecords();

Patient: Fred Ziffle               Age: 56   
Patient: Greg Martell              Age: 56   
Patient: Mary Measles              Age: 47   
Patient: Bob Pointer               Age: 86   
Patient: Lark Manthier             Age: 48   
Patient: Herb Platt                Age: 32   
Patient: Donald Duck               Age: 46   
Patient: John Junior               Age: 26   
Patient: Blaine Little             Age: 92   
Patient: Jane Doe                  Age: 34   
Patient: Martha Stewart            Age: 65   


null

### Patient equality

We as people have an inuitive ability to compare subjectively; a computer, however, does not. Java needs an explicit way to compare the equality of two objects. It does so using hashcodes. As Professor Lanthier's notes state, we define equality by overriding the hashcode method, as the sum of the hashcodes for the attributes we use to define quality:

```java
public int hashCode() {
        return name.hashCode()+((Integer)age).hashCode();
    }
```

We must cast age to an Integer as primitive types do not have hashcodes.

## Takeaways

This is a lot to cover, so let's summarize:

* ArrayLists
    * Flexible lists of data; Arrays, but easier
* Stacks and Queues
    * For when order matters
* HashMaps
    * When you want to access values easily by some associated, known key
    * No duplicate keys; Can have duplicate values
    * Not ordered!
* Sets
    * No duplicates
    * TreeSet for sorted; HashSet for unsorted
* Linked Lists
    * Linear collection of data; kind of like an ArrayList 

Remember not to modify while you iterate!